In [1]:
# Importing NumPy, Pandas, Matplotlib and Seaborn

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
churn_data=pd.read_csv('telecom_churn_data.csv')

In [3]:
churn_data.head()

,mobile_number,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,last_date_of_month_9,arpu_6,...,sachet_3g_9,fb_user_6,fb_user_7,fb_user_8,fb_user_9,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,sep_vbc_3g
0,7000842753,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,197.385,...,0,1.0,1.0,1.0,NaN,968,30.4,0.0,101.20,3.58
1,7001865778,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,34.047,...,0,NaN,1.0,1.0,NaN,1006,0.0,0.0,0.00,0.00
2,7001625959,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,167.690,...,0,NaN,NaN,NaN,1.0,1103,0.0,0.0,4.17,0.00
3,7001204172,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,221.338,...,0,NaN,NaN,NaN,NaN,2491,0.0,0.0,0.00,0.00
4,7000142493,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,261.636,...,0,0.0,NaN,NaN,NaN,1526,0.0,0.0,0.00,0.00


In [4]:
churn_data.shape

(99999, 226)

# Data Preparation

## Missing Value Check

Checking for the columns with only 2 unique values.

In [5]:
# Columns with only 2 unique value.
nunique_2=pd.DataFrame(churn_data.nunique().where(lambda x:x==2).dropna())
churn_data[nunique_2.index].head()

,night_pck_user_6,night_pck_user_7,night_pck_user_8,night_pck_user_9,fb_user_6,fb_user_7,fb_user_8,fb_user_9
0,0.0,0.0,0.0,NaN,1.0,1.0,1.0,NaN
1,NaN,0.0,0.0,NaN,NaN,1.0,1.0,NaN
2,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN


The columns `fb_users_*` and `night_pck_user_*` have only 2 unique values i.e. only 0 & 1. The Null values means that the users are not 
Looking at the above data, we can safely impute 0 for all the missing values for 'fb_users_*' and 'night_pck_user_*'.

In [6]:
# Replacing the Null values with 0.
churn_data[nunique_2.index]=churn_data[nunique_2.index].fillna(0)

Checking for the columns with more than 10% null values.

In [7]:
#Columns with more than 10% null values.
null_series=round(100*(churn_data.isnull().sum(axis=0)/len(churn_data)),2).where(lambda x:x!=0).dropna()
null_greater_10_perc=null_series.sort_values(ascending=False).where(lambda x: x>10).dropna()
null_greater_10_perc

total_rech_data_6           74.85
av_rech_amt_data_6          74.85
arpu_2g_6                   74.85
date_of_last_rech_data_6    74.85
count_rech_3g_6             74.85
arpu_3g_6                   74.85
max_rech_data_6             74.85
count_rech_2g_6             74.85
max_rech_data_7             74.43
count_rech_3g_7             74.43
total_rech_data_7           74.43
count_rech_2g_7             74.43
av_rech_amt_data_7          74.43
arpu_3g_7                   74.43
date_of_last_rech_data_7    74.43
arpu_2g_7                   74.43
date_of_last_rech_data_9    74.08
max_rech_data_9             74.08
arpu_2g_9                   74.08
total_rech_data_9           74.08
av_rech_amt_data_9          74.08
count_rech_2g_9             74.08
arpu_3g_9                   74.08
count_rech_3g_9             74.08
count_rech_2g_8             73.66
max_rech_data_8             73.66
arpu_3g_8                   73.66
date_of_last_rech_data_8    73.66
av_rech_amt_data_8          73.66
total_rech_dat

In [8]:
churn_data[null_greater_10_perc.index].head()

,total_rech_data_6,av_rech_amt_data_6,arpu_2g_6,date_of_last_rech_data_6,count_rech_3g_6,arpu_3g_6,max_rech_data_6,count_rech_2g_6,max_rech_data_7,count_rech_3g_7,...,arpu_3g_9,count_rech_3g_9,count_rech_2g_8,max_rech_data_8,arpu_3g_8,date_of_last_rech_data_8,av_rech_amt_data_8,total_rech_data_8,count_rech_3g_8,arpu_2g_8
0,1.0,252.0,212.17,6/21/2014,1.0,212.17,252.0,0.0,252.0,1.0,...,NaN,NaN,0.0,252.0,212.17,8/8/2014,252.0,1.0,1.0,212.17
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,154.0,0.0,...,NaN,NaN,2.0,25.0,0.00,8/10/2014,50.0,2.0,0.0,7.60
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.84,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,56.0,0.00,6/4/2014,0.0,0.00,56.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Observation:
Here for the columns with more thatn 10% of missing data, we can see that it is Null for all the rows with for a particular month. Also we can see that the missing values are all related to `2g/3g Data`. 

So, we can imply that if a user does not recharge for a data pack, that user's data usage is also expectd to be Null.

Therefore, here we can safely impute 0 for all the above Null values.

Checking for the columns with only 1 unique value.

In [9]:
churn_data[null_greater_10_perc.index]=churn_data[null_greater_10_perc.index].fillna(0)

In [10]:
# Dropping the `date_of_last_rech_data_*` column
churn_data.drop(['date_of_last_rech_data_6',
 'date_of_last_rech_data_7',
 'date_of_last_rech_data_8',
 'date_of_last_rech_data_9'],axis=1,inplace=True)

In [11]:
#Columns with less than 10% null values.
null_series=round(100*(churn_data.isnull().sum(axis=0)/len(churn_data)),2).where(lambda x:x!=0).dropna()
null_less_10_perc=null_series.sort_values(ascending=False).where(lambda x: x<=10).dropna()
null_less_10_perc

loc_og_t2m_mou_9        7.75
roam_ic_mou_9           7.75
loc_og_t2f_mou_9        7.75
loc_ic_t2m_mou_9        7.75
loc_ic_mou_9            7.75
std_og_t2f_mou_9        7.75
loc_og_t2c_mou_9        7.75
std_ic_t2t_mou_9        7.75
loc_ic_t2t_mou_9        7.75
loc_og_t2t_mou_9        7.75
std_ic_t2m_mou_9        7.75
og_others_9             7.75
std_ic_t2f_mou_9        7.75
roam_og_mou_9           7.75
loc_og_mou_9            7.75
spl_og_mou_9            7.75
std_ic_t2o_mou_9        7.75
loc_ic_t2f_mou_9        7.75
onnet_mou_9             7.75
std_og_t2c_mou_9        7.75
std_og_t2t_mou_9        7.75
isd_og_mou_9            7.75
spl_ic_mou_9            7.75
ic_others_9             7.75
offnet_mou_9            7.75
std_og_t2m_mou_9        7.75
std_ic_mou_9            7.75
std_og_mou_9            7.75
isd_ic_mou_9            7.75
std_og_t2c_mou_8        5.38
                        ... 
isd_ic_mou_7            3.86
offnet_mou_7            3.86
std_og_t2f_mou_7        3.86
ic_others_7   

In [12]:
# Removing the columsn with `date` values.
null_less_10_perc=[col for col in null_less_10_perc.index if "date" not in col]

In [13]:
# As we are all dealing with usage based data, a Null value means "No Usage". So can impute 0 for all the Null values.
# Imputing the Null values with '0' of the column for those columns with less than 10% Null values.
churn_data[null_less_10_perc]=churn_data[null_less_10_perc].fillna(0)

In [14]:
# Columns with only 1 unique value.
nunique_1=pd.DataFrame(churn_data.nunique().where(lambda x:x==1).dropna())
churn_data[nunique_1.index].head()

,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,last_date_of_month_9,std_og_t2c_mou_6,std_og_t2c_mou_7,std_og_t2c_mou_8,std_og_t2c_mou_9,std_ic_t2o_mou_6,std_ic_t2o_mou_7,std_ic_t2o_mou_8,std_ic_t2o_mou_9
0,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
cols=[col for col in churn_data.columns if "date" in col]
cols

['last_date_of_month_6',
 'last_date_of_month_7',
 'last_date_of_month_8',
 'last_date_of_month_9',
 'date_of_last_rech_6',
 'date_of_last_rech_7',
 'date_of_last_rech_8',
 'date_of_last_rech_9']

In [16]:
# Dropping the columns with date.
churn_data=churn_data.drop(cols,axis=1)

In [17]:
churn_data.shape

(99999, 214)

## Deriving New Variables

##### Call and Data Usage Drop
There are 3 phases named as: `Good Phase`(6th and 7th month), `Action Phase`(8th month) & `Churn Phase`(9th month).

If we see a drop in call or data usage from `Good` to `Action` Phase, it can be a good indicator that the customer is not happy with the service and is about to churn.

So, we are deriving a new variable which will 1 when we see a drop in usage by more than 50% else 0.

In [18]:
# Creating the average CALL usage(both incomming and outgoing) for "Good Phase".
churn_data['good_phase_call']=(churn_data['total_ic_mou_6']+churn_data['total_og_mou_6']+churn_data['total_ic_mou_7']+churn_data['total_og_mou_7'])/2

In [19]:
# Creating the average DATA usage(both 2G and 3G) for "Good Phase".
churn_data['good_phase_data']=(churn_data['vol_2g_mb_6']+churn_data['vol_3g_mb_6']+churn_data['vol_2g_mb_7']+churn_data['vol_3g_mb_7'])/2

In [20]:
# Creating the CALL usage(both incomming and outgoing) for "Action Phase".
churn_data['action_phase_call']=churn_data['total_ic_mou_8']+churn_data['total_og_mou_8']

In [21]:
# Creating the CALL usage(both 2G and 3G) for "Action Phase".
churn_data['action_phase_data']=churn_data['vol_2g_mb_8']+churn_data['vol_3g_mb_8']

In [22]:
churn_data['diff_call_bet_action_and_good_phase']=churn_data['action_phase_call']-churn_data['good_phase_call']

In [23]:
churn_data['diff_data_bet_action_and_good_phase']=churn_data['action_phase_data']-churn_data['good_phase_data']

In [24]:
filter1=churn_data['diff_call_bet_action_and_good_phase']<0
filter2=abs(churn_data['diff_call_bet_action_and_good_phase'])>=(0.5*churn_data['good_phase_call'])
churn_data.loc[filter1 & filter2,"call_usage_drop"]=1
churn_data['call_usage_drop']=churn_data['call_usage_drop'].fillna(0)

In [25]:
churn_data[['good_phase_call','action_phase_call','diff_call_bet_action_and_good_phase','call_usage_drop']].head()

,good_phase_call,action_phase_call,diff_call_bet_action_and_good_phase,call_usage_drop
0,0.000,5.44,5.440,0.0
1,174.950,735.72,560.770,0.0
2,728.920,680.49,-48.430,0.0
3,427.615,405.34,-22.275,0.0
4,391.670,546.97,155.300,0.0


In [26]:
filter1=churn_data['diff_data_bet_action_and_good_phase']<0
filter2=abs(churn_data['diff_data_bet_action_and_good_phase'])>=(0.5*churn_data['good_phase_data'])
churn_data.loc[filter1 & filter2,"data_usage_drop"]=1
churn_data['data_usage_drop']=churn_data['data_usage_drop'].fillna(0)

In [27]:
churn_data[['good_phase_data','action_phase_data','diff_data_bet_action_and_good_phase','data_usage_drop']].head()

,good_phase_data,action_phase_data,diff_data_bet_action_and_good_phase,data_usage_drop
0,132.890,115.36,-17.530,0.0
1,54.035,365.47,311.435,0.0
2,0.000,0.00,0.000,0.0
3,0.000,0.00,0.000,0.0
4,0.000,0.00,0.000,0.0


In [28]:
# Dropping the intermediate columns
churn_data.drop(['good_phase_call','good_phase_data','action_phase_call','action_phase_data','diff_call_bet_action_and_good_phase','diff_data_bet_action_and_good_phase'],axis=1,inplace=True)

##### High Roaming Users
Creating a new categorical variable `high_roam_user`, which has 1 for the customers who use more than 90 percentile of sum of roaming minutes_of_usage.

In [29]:
# Calculating the 90 percentile value for the sum of roaming minutes_of_usage.
x=churn_data[['roam_ic_mou_6','roam_og_mou_6','roam_ic_mou_7','roam_og_mou_7','roam_ic_mou_8','roam_og_mou_8']].sum(axis=1).quantile(0.9)
x

110.66199999999998

In [30]:
# Number of High Roaming Users(who use more than 90 percentile of sum of roaming minutes_of_usage)
churn_data['mobile_number'].where(churn_data[['roam_ic_mou_6','roam_og_mou_6','roam_ic_mou_7','roam_og_mou_7','roam_ic_mou_8','roam_og_mou_8']].sum(axis=1)>x).dropna().shape

(10000,)

In [31]:
# Creating new column "high_roam_user".
churn_data.loc[churn_data[['roam_ic_mou_6','roam_og_mou_6','roam_ic_mou_7','roam_og_mou_7','roam_ic_mou_8','roam_og_mou_8']].sum(axis=1)>x,"high_roam_user"]=1
churn_data['high_roam_user'].fillna(0,inplace=True)

## Selecting the `High Value Customers`

In [32]:
# Checking for the columns with missing values
# Calculating the percentage of Null values present in each columns and storing it in a list.
null_series=round(100*(churn_data.isnull().sum(axis=0)/len(churn_data)),2).where(lambda x:x!=0).dropna()
print('No. of Columns having Null values = ',null_series.count())

No. of Columns having Null values =  0


In [33]:
churn_data['total_6']=churn_data['total_rech_amt_6']+(churn_data['total_rech_data_6']*churn_data['av_rech_amt_data_6'])
churn_data['total_7']=churn_data['total_rech_amt_7']+(churn_data['total_rech_data_7']*churn_data['av_rech_amt_data_7'])

In [34]:
#Calculating the average Recharge for month 6 and 7.
churn_data['avg']=(churn_data['total_6']+churn_data['total_7'])/2

In [35]:
# Dropping the intermediate columns
churn_data.drop(['total_6','total_7'],axis=1,inplace=True)

In [36]:
# Calculating the 70th Percentile of average recharge amount in the first two months
avg=churn_data['avg'].quantile(0.70)
print('70th percentile of the average recharge amount in the first two months = ',avg)

70th percentile of the average recharge amount in the first two months =  478.0


In [37]:
# Number of High Value Customer(who recharge more than 70 percentile of average)
churn_data['mobile_number'].where(churn_data['avg']>avg).dropna().shape

(29953,)

In [38]:
# Cretaing a new dataset for High Value Customer only
churn_data_new=churn_data[churn_data['mobile_number'].where(churn_data['avg']>avg).isnull() == False]
churn_data_new.shape

(29953, 218)

## Tagging the `Churners`

Conditions: For the rows where all the below columns have 0, tag them as `Churners`.

total_ic_mou_9 == 0

total_og_mou_9 == 0

vol_2g_mb_9 == 0

vol_3g_mb_9 == 0

In [39]:
# Setting the conditions
filter1=churn_data_new['total_ic_mou_9']==0 
filter2=churn_data_new['total_og_mou_9']==0 
filter3=churn_data_new['vol_2g_mb_9']==0 
filter4=churn_data_new['vol_3g_mb_9']==0

In [40]:
churn_data_new.loc[filter1 & filter2 & filter3 & filter4,'churn']=1

C:\Users\USER.000\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\USER.000\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [41]:
churn_data_new['churn'].fillna(0,inplace=True)

C:\Users\USER.000\Anaconda3\lib\site-packages\pandas\core\generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [42]:
print("No. of Churners = ",churn_data_new['churn'].sum())

No. of Churners =  2433.0


In [43]:
churn_data_new['churn'].value_counts()

0.0    27520
1.0     2433
Name: churn, dtype: int64

#### Observations:

Here we can see that there are only 2433 Churned customers(~8% of High Value Customers) and 27520 Non-Churned Customers(~92% of High Value Customer data).

This is a highly imbalanced dataset.

### Dropping the columns related to 9th month.

In [44]:
# Dropping the columns with '_9'.
cols=[col for col in churn_data_new.columns if "_9" in col]
print("Number of columns with _9 as part of it = ",len(cols))

Number of columns with _9 as part of it =  51


In [45]:
churn_data_new=churn_data_new.drop(cols,axis=1)

In [46]:
churn_data_new.shape

(29953, 168)

### Checking & Removing all the columns with only 1 unique values

In [47]:
# Columns with only 1 unique value.
nunique_1=pd.DataFrame(churn_data_new.nunique().where(lambda x:x==1).dropna())
nunique_1.index

Index(['circle_id', 'loc_og_t2o_mou', 'std_og_t2o_mou', 'loc_ic_t2o_mou',
       'std_og_t2c_mou_6', 'std_og_t2c_mou_7', 'std_og_t2c_mou_8',
       'std_ic_t2o_mou_6', 'std_ic_t2o_mou_7', 'std_ic_t2o_mou_8'],
      dtype='object')

In [48]:
churn_data_new[nunique_1.index].head()

,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,std_og_t2c_mou_6,std_og_t2c_mou_7,std_og_t2c_mou_8,std_ic_t2o_mou_6,std_ic_t2o_mou_7,std_ic_t2o_mou_8
0,109,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,109,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,109,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21,109,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23,109,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


As we can see that apart from `circle_id`, all the other columns with only 1 unique value have `0`. 

Also as with no variance, it will not contribute to further analysis. So, we can drop them.

In [49]:
churn_data_new.drop(nunique_1.index,axis=1,inplace=True)

In [50]:
churn_data_new.shape

(29953, 158)

In [51]:
cols=[col for col in churn_data_new.columns if ("og" in col) and ("_6" in col) and ("total_og" not in col)]
cols

['roam_og_mou_6',
 'loc_og_t2t_mou_6',
 'loc_og_t2m_mou_6',
 'loc_og_t2f_mou_6',
 'loc_og_t2c_mou_6',
 'loc_og_mou_6',
 'std_og_t2t_mou_6',
 'std_og_t2m_mou_6',
 'std_og_t2f_mou_6',
 'std_og_mou_6',
 'isd_og_mou_6',
 'spl_og_mou_6',
 'og_others_6']

In [52]:
from sklearn.model_selection import train_test_split

# To make sure that the train and test data set always have the same rows, respectively
x_train, x_test = train_test_split(churn_data_new, train_size = 0.7, test_size = 0.3, random_state = 100)

In [53]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [54]:
# Apply scaler() to all the columns except the 'dummy' variables
num_vars = ['symboling','cylinder_number','wheelbase','carlength','carwidth','carheight','curbweight','enginesize','boreratio','stroke','compressionratio','horsepower','peakrpm','citympg','highwaympg','price']

carprice_train[num_vars] = scaler.fit_transform(carprice_train[num_vars])

carprice_train.head()

NameError: name 'carprice_train' is not defined

In [ ]:
#Importing the PCA module
from sklearn.decomposition import PCA
pca = PCA( random_state=100)

In [ ]:
#Doing the PCA on the train data
pca.fit(x_train)

In [ ]:
#Let's check the components
pca.components_

In [ ]:
components = pd.DataFrame({'PC1':pca.components_[0],'PC2':pca.components_[1],'Feature':X.columns })
components